In [1]:
import pandas as pd
import numpy as np
from numpy.linalg import inv
from numpy.linalg import svd
from scipy.optimize import linprog

In [2]:
import matlab.engine
eng = matlab.engine.start_matlab()

function Num_Stab_Approx

RM=6
penalty= 7
normalised= 1
X(1:T-1,:)= trial 4 datax
y(1:T-1,:)=trial 4 datay
X is generated assuming d=1

In [2]:
test = pd.read_csv("trial4_x.csv", encoding = "ISO-8859-1")
add=test.columns.tolist()
test.loc[-1] = add 
test.index = test.index + 1  
test= test.sort_index()
test.columns=["blah","blah2","blah3"]
x = np.copy(test)

test = pd.read_csv("trial4_y.csv", encoding = "ISO-8859-1")
add=test.columns.tolist()
test.loc[-1] = add 
test.index = test.index + 1  
test= test.sort_index()
test.columns=["blah"]
y= np.copy(test)
y[0] = float(y[0])
x[0] = [1., 48.29917764394355828, 1.]

Function begin

In [3]:
RM = 6
penalty= 7 
normalised= 1

Function body

In [4]:
T = x.shape[0]
n = x.shape[1]
N = y.shape[1] #Compute the number of columns in Y

In [ ]:
if normalised == 1 or RM >= 5:
    
    X1 = np.divide(
    (x[:,1:n] - np.matmul(np.ones((T, 1)), x[:,1:n].mean(axis=0).reshape(1, n-1))),
    np.matmul(np.ones((T, 1)), np.array([np.std(x[:,i], ddof=1) for i in range(1,n)]).reshape(1,n-1))
    )
    X1 = X1.astype(float)
    #unfortunately np.std seems cannot return an array for each column, therefore I was forced to use a list comprehension here
    
    Y1 = np.divide(
    (y - np.ones((T,1))*np.mean(y)),
    np.matmul(np.ones((T,1)), np.std(y, ddof=1).reshape(1,1))
    )
    Y1 = Y1.astype(float)
    n1 = n-1 # Number of coefficients in a regression with normalized data is reduced by 1 (no intercept)
else:

    X1 = x
    Y1 = Y          # Leave Y without changes
    n1 = n

In [5]:
X1 = np.divide(
    (x[:,1:n] - np.matmul(np.ones((T, 1)), x[:,1:n].mean(axis=0).reshape(1, n-1))),
    np.matmul(np.ones((T, 1)), np.array([np.std(x[:,i], ddof=1) for i in range(1,n)]).reshape(1,n-1))
    ) 
X1 = X1.astype(float)
    #unfortunately np.std seems cannot return an array for each column, therefore I was forced to use a list comprehension here
Y1 = np.divide(
    (y - np.ones((T,1))*np.mean(y)),
    np.matmul(np.ones((T,1)), np.std(y, ddof=1).reshape(1,1))
    )
Y1 = Y1.astype(float)
n1 = n-1 # Number of coefficients in a regression with normalized data is reduced by 1 (no intercept)

In [54]:
# 3. Regression methods
#simple OLS
if RM == 1:
    B = np.matmul(
        inv(np.matmul(X1.transpose(),X1).astype(float)), #need to convert the output to float or else return error
        np.matmul(X1.transpose(),Y1)
        )
#LS-SVD
elif RM == 2:
    U, S, V = svd(X1, full_matrices=False) #The minus is in the opposite side compared to matlab, but should not have a difference in result
    S_inv = np.diag(1/S)
    B = np.matmul(np.matmul(np.matmul(V,S_inv),U.transpose()),Y1) #The B will have the same value but will be upside down compared to matlab as a result

#check if changes result or not###

#LAD-PP
elif RM == 3:
    
    # #This method is strongly discouraged! It is one of the slowest among all other regression
    # #construct the boundaries
    BND = [(-100, 100)]*n1 + [(0, float("inf"))]*2*T
    f = [0]*n1+ [1]*2*T
    #specify the Aeq an beq
    Aeq = np.concatenate((X1, np.identity(T), -np.identity(T)), axis=1)
    B =[]
    #solve the equation
    for j in range(N):
        beq = Y1[:,j].tolist()
        result = linprog(f, A_eq = Aeq, b_eq = beq, bounds= BND , method='revised simplex')
        B.append(result.x) 
    
    #Or so it should. Unfortunately, the Scipy package will took ages to even finish computing one optimisation solution given the size of Aeq and beq. Without other choices, I import the function that does this part of computation from matlab, which is able to finish the computation within seconds.

# LAD-DP
elif RM == 4:
#Possibily also very slow
#Define Boundary
BND = [(-1,1)]*T

#Define aeq and beq for linprog
Aeq = X1.transpose()
beq = np.zeros((n1, 1))
B = []
for i in range(N):
    f = -Y1[:,i]
    lamda = linprog(f, A_eq = Aeq, b_eq = beq, bounds= BND)
    B.append(lamda)

#RLS-Tikhonov
elif RM == 5:

# RLS-TSVD
elif RM == 6:


In [82]:
U, S, V = svd(X1, full_matrices=False) #The minus is in the opposite side compared to matlab, but should not have a difference in result
S_inv = np.diag(1/S)
B = np.matmul(np.matmul(np.matmul(V,S_inv),U.transpose()),Y1) #The B will have the same value but will be upside down compared to matlab as a result

#check if changes result or not###

test tmr

In [7]:
#rm == 4
#Define Boundary
BND = [(-1,1)]*T

#Define aeq and beq for linprog
Aeq = X1.transpose()
beq = np.zeros((n1, 1))
B = []
for i in range(N):
    f = -Y1[:,i]
    lamda = linprog(f, A_eq = Aeq, b_eq = beq, bounds= BND)
    B.append(lamda)

In [3]:
res.x

array([ 9.99999989, -2.99999999])